# SP3 Files and Orbit Interpolation

To process DORIS data we need to be able to compute (or estimate) the satellite position at the measurement epochs. The most simple case would be to interpolate satellite coordinates at every requested epoch, using the tabulated position (an/or velocity) vectors contained in a high-quality [Sp3](https://files.igs.org/pub/data/format/sp3c.txt) file.


## Sp3 Files

sp3 files contain tabulated satellite coordinates and (optionally) velocity and onboard clock correction estimates at selected epochs. Before extracting/using the data, we need to know at which spatial and temporal frame the coordinates and timestamps are aligned to. This information, along with other metadata, are recorded in the file header. The temporal resolution of the data may differ for different files.

Here is an example of an sp3c header:
```text
#cV2023 12 23  1 20  0.00000000   14307 ORBIT ITRF  FIT CNES
## 2293 523200.00000000    60.00000000 60301 0.0555555555556
+    1   L40  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
+          0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
+          0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
+          0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
+          0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
++
++
++         0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
++         0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
++         0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
%c L  cc TAI ccc cccc cccc cccc cccc ccccc ccccc ccccc ccccc
%c cc cc ccc ccc cccc cccc cccc cccc ccccc ccccc ccccc ccccc
%f  1.2500000  1.025000000  0.00000000000  0.000000000000000
%f  0.0000000  0.000000000  0.00000000000  0.000000000000000
%i    0    0    0    0      0      0      0      0         0
%i    0    0    0    0      0      0      0      0         0
/* CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
/* CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
/* CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
/* CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
```

Example of an sp3c data segment, containing tabultaed satellite position and velocity for three individual epochs.
```text
*  2023 12 23  1 20  0.00000000
PL40   3424.775359   -633.803923  -6885.685565 999999.999999
VL40 -16166.218860  66160.075712 -14123.204711 999999.999999
*  2023 12 23  1 21  0.00000000
PL40   3324.272067   -235.633419  -6959.643057 999999.999999
VL40 -17325.676895  66530.896615 -10522.932928 999999.999999
*  2023 12 23  1 22  0.00000000
PL40   3216.979432    164.175072  -7011.895627 999999.999999
VL40 -18429.043288  66705.854052  -6890.097734 999999.999999
```

Example of an sp3c data segment, containing tabultaed satellite position and clock corrections for a micture of GNSS satellites.
```text
*  2024  4 14  0  0  0.00000000
PG02  -5202.216712  17102.556431 -19068.134475   -453.815651
PG03   7702.033967  19962.377229 -15966.731346    309.184457
[...]
PR03  -3266.003590 -24846.578198   4964.088241     98.202172
PR04  -9997.719190 -20284.322613 -11775.755002    231.903331
[...]
PE02  10890.276798 -12805.372335 -24373.530624    119.253409
PE03  12786.435328 -23973.892572  11728.972306   -107.029295
[...]
PC06 -16203.734845  30645.557501  23878.014078    344.408313
PC07  -6271.375391  38817.335315  15594.731253   -823.067866
[...]
PJ02 -25889.506821  22597.603383  29500.349114     -4.030198
```

## Interpolating Satellite State(*) from Sp3

The most rigorous way of interpolating satellite state at a given epoch, given that we already have initial conditions at some other instant in time, would be to solve the equations of motion of the satellite. However, this would involve a cumbersome computation of accelerations induced by every force acting upon the satellite and the involvement of a numerical differential equation solver. Given the fact that we already poccess a pretty dense temporal resolution (e.g. 60sec for most LEO) via the sp3 files and the fact that accuraccy demands are not the highest for the range of applicatins to be considered, we can interpolate satellite state using the sp3 values.

To interpolate sp3 files we will need the `Sp3Interpolator` class of the `dsoclasses.orbits.interpolator` module. We construct an instance of the class, using 
```python
Sp3Interpolator(sp3fn, sat_systems, interval_in_sec=1800, min_data_pts=4, itype="Polynomial", exclude_missing_clock_values=False, exclude_flag_events=[])
```
where:
    - `sp3fn` is the name of the sp3 file, including path,
    - `sat_systems` is a list of satellite systems to be considered (e.g. 'G' for GPS, 'R for Glonass, etc). 'L' is usually used for LEOs,
    - `interval_in_sec` specifies the interval used to interpolate. E.g. given an interpolation epoch $t$, we will consider all data points that lay within `interval_in_sec` seconds away from $t$,
    - `min_data_points` specifies the minimum number of data points to be used for the interpolation,
    - `itype` is the interpolation algorithm to use; it can be any of 'Polynomial', 'CubicSpline', or [PchipInterpolator](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.PchipInterpolator.html),
    - `exclude_missing_clock_values` signals the function to exclude any record that is missing a clock correction, and
    - `exclude_flag_events` signals the function to exclude any record when some specific event has been recorded

Once the instance is constructed, we can query satellite position and clock corrections using a call to 
```python
xsat, ysat, zsat, clk = Sp3Interpolator.sat_at(satellite_id, t)
```

(*) **state** is the concatenations of the position plus velocity vector, i.e.
$$
\vec{s} = \begin{pmatrix} \vec{r} & \vec{v} \end{pmatrix}^T
$$

In [2]:
from dsoclasses.orbits.interpolator import Sp3Interpolator
from os.path import join
import datetime

data_path = "/home/xanthos/Software/AcademicSoftware/data"
dsp3 = join(data_path, "ssas6a20.b23357.e24001.DG_.sp3.001")

# create an Sp3Interpolator instance
intrp = Sp3Interpolator.from_sp3(dsp3, ['L'],interval_in_sec=1800,min_data_pts=12,itype='CubicSpline',exclude_missing_clock_values=False,exclude_flag_events=['M', 'E'])

# get the list of satellites available
# print(f'Sp3Interpolator includes the following satellites: {intrp.sat_list()}')

# get the time system of the interpolator
# print(f'Sp3Interpolator\'s time system is {intrp.time_system()}')

x, y, z, clk = intrp.sat_at('L40', datetime.datetime(2024,1,1,12,0,30))
print(f'Satellite position at {datetime.datetime(2024,1,1,12,0,30)} is {x:.3f}, {y:.3f}, {z:.3f} [m]')

TypeError: isinstance() arg 2 must be a type, a tuple of types, or a union

### Example: Approximate Interpolation Error

The sp3 file `ssas6a20.b23357.e24001.DG_.sp3.001` contains CNES computed orbit for the satellite Sentinek-6a (i.e. 'L40') with a frequency of 60 sec. To approximate the orbit interpolation error, we have removed from the file the record for the date 2024-01-01, at 00:00:00 while very other record is left as is. We are now going to interpolate this missing record and compare it with the one recorded in the original file to approximate the interpolation error.
Thus, the (original) file ssas6a20.b23357.e24001.DG_.sp3.001 contains the following records, while `ssas6a20.missing.data.sp3.001` does not:
```text
*  2024  1  1  0  0  0.00000000                                                 
PL40  -2806.014690  -1814.460651   6953.187645 999999.999999                    
VL40  16419.732661 -66738.749528 -10780.441836 999999.999999
```

In [35]:
data_path = "/home/xanthos/Software/AcademicSoftware/data"
t_missing = datetime.datetime(2024,1,1,0,0,0)

# First use the original file to interpolate at the epoch of request. 
# Obviously, the result should be the record itsef!

dsp3 = join(data_path, "ssas6a20.b23357.e24001.DG_.sp3.001")
intrp = Sp3Interpolator(dsp3, ['L'], 1800, 12, 'CubicSpline', False, ['M', 'E'])
xref, yref, zref, clk = intrp.sat_at('L40', t_missing)
print(f'Reference satellite position at {t_missing} is {xref:.3f}, {yref:.3f}, {zref:.3f} [m]')

# now use the file with the missing epoch to approximate error
dsp3 = join(data_path, "ssas6a20.missing.data.sp3.001")

# Test 1:
intrp = Sp3Interpolator(dsp3, ['L'], 1800, 12, 'CubicSpline', False, ['M', 'E'])
x, y, z, clk = intrp.sat_at('L40', t_missing)
print(f'Test 1: diffs at {t_missing} w.r.t reference coordinates: {(x-xref):.3f}, {(y-yref):.3f}, {(z-zref):.3f} [m]')

# Test 2:
intrp = Sp3Interpolator(dsp3, ['L'], 900, 12, 'CubicSpline', False, ['M', 'E'])
x, y, z, clk = intrp.sat_at('L40', t_missing)
print(f'Test 2: diffs at {t_missing} w.r.t reference coordinates: {(x-xref):.3f}, {(y-yref):.3f}, {(z-zref):.3f} [m]')

# Test 3:
intrp = Sp3Interpolator(dsp3, ['L'], 300, 8, 'CubicSpline', False, ['M', 'E'])
x, y, z, clk = intrp.sat_at('L40', t_missing)
print(f'Test 3: diffs at {t_missing} w.r.t reference coordinates: {(x-xref):.3f}, {(y-yref):.3f}, {(z-zref):.3f} [m]')

# Test 4:
intrp = Sp3Interpolator(dsp3, ['L'], 241, 6, 'CubicSpline', False, ['M', 'E'])
x, y, z, clk = intrp.sat_at('L40', t_missing)
print(f'Test 4: diffs at {t_missing} w.r.t reference coordinates: {(x-xref):.3f}, {(y-yref):.3f}, {(z-zref):.3f} [m]')

# Test 5:
intrp = Sp3Interpolator(dsp3, ['L'], 181, 6, 'CubicSpline', False, ['M', 'E'])
x, y, z, clk = intrp.sat_at('L40', t_missing)
print(f'Test 5: diffs at {t_missing} w.r.t reference coordinates: {(x-xref):.3f}, {(y-yref):.3f}, {(z-zref):.3f} [m]')

# Test 6:
intrp = Sp3Interpolator(dsp3, ['L'], 121, 4, 'CubicSpline', False, ['M', 'E'])
x, y, z, clk = intrp.sat_at('L40', t_missing)
print(f'Test 6: diffs at {t_missing} w.r.t reference coordinates: {(x-xref):.3f}, {(y-yref):.3f}, {(z-zref):.3f} [m]')

Reference satellite position at 2024-01-01 00:00:00 is -2806014.690, -1814460.651, 6953187.645 [m]
Test 1: diffs at 2024-01-01 00:00:00 w.r.t reference coordinates: 0.379, 0.845, -4.693 [m]
Test 2: diffs at 2024-01-01 00:00:00 w.r.t reference coordinates: 0.379, 0.845, -4.693 [m]
Test 3: diffs at 2024-01-01 00:00:00 w.r.t reference coordinates: 0.376, 0.837, -4.647 [m]
Test 4: diffs at 2024-01-01 00:00:00 w.r.t reference coordinates: 0.396, 0.881, -4.887 [m]
Test 5: diffs at 2024-01-01 00:00:00 w.r.t reference coordinates: 0.340, 0.759, -4.214 [m]
Test 6: diffs at 2024-01-01 00:00:00 w.r.t reference coordinates: 0.909, 2.023, -11.221 [m]


## DORIS sp3 Products

sp3 products derived from or including DORIS observations are publicly available at [CDDIS](https://cddis.nasa.gov/archive/doris/products/orbits/) (now [Earthdata](https://www.earthdata.nasa.gov/data/space-geodesy-techniques/doris/international-doris-service-orbit-product)) webpage.